In [1]:
import numpy as np
import math
import scipy.optimize as optimize
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

def calibrate3DSensorInUniformField(samples_x,samples_y,samples_z):
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(samples_x, samples_y, samples_z)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()
    def residualfunc(params):
        return (x-params[0])**2/params[3]**2+(y-params[1])**2/params[4]**2+(z-params[2])**2/params[5]**2-1
    params=optimize.leastsq(residualfunc,[0,0,0,1,1,1])
    params=params[0]
    offsets=[params[0],params[1],params[2]]
    amplitudes=[params[3],params[4],params[5]]
    print("Offset x = ", offsets[0])
    print("Offset y = ", offsets[1])
    print("Offset z = ", offsets[2])
    print("Amplitude x = ", amplitudes[0])
    print("Amplitude y = ", amplitudes[1])
    print("Amplitude z = ", amplitudes[2])
    return offsets, amplitudes

def noiseRMSmeasure(gen_fun,Ts,n):
    print("Noise measurement ongoing...")
    time.sleep(1)
    samples=np.zeros((n,len(gen_fun())))
    for i in range(n):
        samples[i]=gen_fun()
        time.sleep(Ts)
    RMSnoise=np.std(samples,axis=0)
    print("RMS Noise (1sigma) = ",RMSnoise)
    print("Done")
    return RMSnoise
        
def getScatteredSamples(gen_fun,dist,n):
    print("Get ready...")
    print("3")
    time.sleep(1)
    print("2")
    time.sleep(1)
    print("1")
    time.sleep(1)
    print("GO!")
    samples=np.zeros((n,len(gen_fun())))
    i=0
    while i<n:
        new_sample=gen_fun()
        norms=np.linalg.norm(samples-new_sample,axis=1)
        if all(norms>dist):
            print("Sample ",i+1,"/",n," = ",new_sample)
            samples[i][:]=new_sample
            i=i+1
        time.sleep(0.01)
    print("Done")
    return samples
    

    
if __name__ == '__main__':
    #3D calibration
    theta=np.arange(0,2*math.pi,0.1)
    phi=np.arange(0,math.pi,math.pi/4)
    x0,y0,z0=-20, 2, -10
    a=50
    b=13
    c=12
    theta, phi = np.meshgrid(theta,phi)
    def ellipsoid(theta,phi,a,b,c):
        return a*np.multiply(np.sin(theta),np.cos(phi)), b*np.multiply(np.sin(theta),np.sin(phi)), c*np.cos(theta)
    x,y,z=ellipsoid(theta,phi,a,b,c)
    x=np.ndarray.flatten(x)+x0
    y=np.ndarray.flatten(y)+y0
    z=np.ndarray.flatten(z)+z0
    calibrate3DSensorInUniformField(x,y,z)
    
    #Noise measure
    from random import random
    def sample_generator():
        return np.array([random(), random(), random()])
    noiseRMSmeasure(sample_generator,0.001,100)
    
    #Scattered sampling
    getScatteredSamples(sample_generator,0.2,10)
    

<Figure size 640x480 with 0 Axes>

Offset x =  -20.0
Offset y =  2.0
Offset z =  -10.0
Amplitude x =  50.0
Amplitude y =  13.0
Amplitude z =  12.0
Noise measurement ongoing...
RMS Noise (1sigma) =  [0.28562257 0.28690494 0.29289312]
Done
Get ready...
3
2
1
GO!
Sample  1 / 10  =  [0.49275117 0.89259932 0.58775553]
Sample  2 / 10  =  [0.94359573 0.83473595 0.94977409]
Sample  3 / 10  =  [0.69098313 0.78698833 0.70078541]
Sample  4 / 10  =  [0.11836743 0.50844519 0.0583912 ]
Sample  5 / 10  =  [0.14128702 0.31330367 0.14531023]
Sample  6 / 10  =  [0.72485805 0.54872358 0.79073367]
Sample  7 / 10  =  [0.33978451 0.16270574 0.94482831]
Sample  8 / 10  =  [0.79832948 0.92149858 0.08191264]
Sample  9 / 10  =  [0.15847234 0.88713883 0.58297574]
Sample  10 / 10  =  [0.47842737 0.59939219 0.75946776]
Done
